In [6]:
# %pip install --upgrade --quiet langchain langchain-community chromadb bs4 boto3
# %pip install --upgrade --quiet pydantic
%pip install --upgrade --quiet sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install --upgrade --quiet trulens_eval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [26]:
# %pip install --quiet "ipython>=8.12.0" "ipywidgets>=8.0.6"

In [2]:
%pip show langchain

Name: langchain
Version: 0.1.14
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/ec2-user/miniconda3/envs/llm_notebook_env/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [1]:
from typing import Dict
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json

sagemaker_endpoint_name = "mt-djl-ds-chatglm3-g4dn"

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        parameters = model_kwargs
        if 'temperature' not in parameters:
            parameters['temperature'] = 0.01
        input = {"inputs": prompt, "parameters": parameters }
        input_str = json.dumps(input).encode('utf-8')
        return input_str
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["outputs"]

chatglm_model = SagemakerEndpoint(
    endpoint_name=sagemaker_endpoint_name, 
    region_name="us-east-1", 
    content_handler=ContentHandler()
)


In [2]:
chatglm_model.invoke("Amazon Bedrock是什么?")

'Amazon Bedrock 是一种基于云的基础设施即服务(IaaS)工具，旨在为开发人员和运维团队提供一种可靠、可扩展的基础设施管理方式。它提供了一个统一的管理界面，让用户可以轻松管理 AWS 资源，包括计算实例、存储、数据库、网络等等。\n\nAmazon Bedrock 的设计目标是简化 AWS 基础设施的管理，同时提供高可用性、可靠性和性能。它使用自动化工具和脚本来管理基础设施，并使用机器学习来自动检测和预测潜在的问题。此外，Amazon Bedrock 还提供了一种名为“基础设施即代码”(Infrastructure as Code)的方式，让用户可以将基础设施配置和代码化，以便更好地管理和版本控制。\n\n总的来说，Amazon Bedrock 提供了一种简单、高效、可靠的方式，帮助用户管理 AWS 基础设施，并提高了运维效率。'

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='aspire/acge_text_embedding')

In [4]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://huggingface.co/blog/zh/moe")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory="./chroma_db_500chunk/")

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [5]:
questions = [
    "MOE有什么优势?",
    "为什么MOE的训练和推理的速度更快?",
    "简单总结MoE模型的特点?",
    "Shazeer 将 MoE 应用在了哪个应用领域?",
    "MoE模型的训练过程中，如何保证令牌不会总是被发送给少数几个受欢迎的专家?",
    "MoE模型训练时，怎么保证token的负载均衡?",
    "GShard如何保证MoE模型的负载均衡?",
    "MoE的专家数量达到多少以后，效率就会显著降低?",
    "怎么将MoE模型的推理阶段更加高效?",
    "在微调MoE这样的稀疏模型时，怎么防止出现过拟合?",
    "模型的并行有几种形式?",
    "部署 MoE 模型有哪些技术?"
]

In [6]:
results = vectorstore.similarity_search(questions[0], k=2)
results

[Document(page_content='组件专家: 在传统的 MoE 设置中，整个系统由一个门控网络和多个专家组成。在支持向量机 (SVMs) 、高斯过程和其他方法的研究中，MoE 通常被视为整个模型的一部分。然而，Eigen、Ranzato 和 Ilya 的研究 探索了将 MoE 作为更深层网络的一个组件。这种方法允许将 MoE 嵌入到多层网络中的某一层，使得模型既大又高效。\n条件计算: 传统的神经网络通过每一层处理所有输入数据。在这一时期，Yoshua Bengio 等研究人员开始探索基于输入令牌动态激活或停用网络组件的方法。', metadata={'description': 'We’re on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.', 'source': 'https://huggingface.co/blog/zh/moe', 'title': '混合专家模型（MoE）详解'}),
 Document(page_content='组件专家: 在传统的 MoE 设置中，整个系统由一个门控网络和多个专家组成。在支持向量机 (SVMs) 、高斯过程和其他方法的研究中，MoE 通常被视为整个模型的一部分。然而，Eigen、Ranzato 和 Ilya 的研究 探索了将 MoE 作为更深层网络的一个组件。这种方法允许将 MoE 嵌入到多层网络中的某一层，使得模型既大又高效。\n条件计算: 传统的神经网络通过每一层处理所有输入数据。在这一时期，Yoshua Bengio 等研究人员开始探索基于输入令牌动态激活或停用网络组件的方法。', metadata={'description': 'We’re on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.', 'source': 'https://huggingface.c

In [7]:
len(splits)

42

In [8]:
from langchain.prompts import PromptTemplate

prompt_template = """请基于以下的3引号里面的内容，简洁、准确的回答最后的问题，并使用中文。如果你无法从已知内容获取答案，就直接返回'根据已知信息无法回答该问题。'，不要编造答案。

已知内容:
```
{context}
```

问题: {question}
答案:"""


In [9]:
prompt_template.format(question='qqqqq', context= 'xcccccc')

"请基于以下的3引号里面的内容，简洁、准确的回答最后的问题，并使用中文。如果你无法从已知内容获取答案，就直接返回'根据已知信息无法回答该问题。'，不要编造答案。\n\n已知内容:\n```\nxcccccc\n```\n\n问题: qqqqq\n答案:"

In [10]:
from trulens_eval import Tru

# initialize evaluation db
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [11]:
from trulens_eval.tru_custom_app import instrument

class RAG_App:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vectorstore.similarity_search(query, k=5)
        docs = [doc.page_content for doc in results]
        return docs

    @instrument
    def generate_completion(self, query: str, context: list) -> str:
        """
        Generate answer from context.
        """
        context_str = "\n\n".join(context)
        p = prompt_template.format(question=query, context=context_str)
        return chatglm_model(p)

    @instrument
    def query(self, query: str) -> str:
        context_slist = self.retrieve(query)
        completion = self.generate_completion(query, context_slist)
        return completion

rag = RAG_App()

In [12]:
from trulens_eval import Feedback, Select, Bedrock
from trulens_eval.feedback import Groundedness
import numpy as np

fclaude = Bedrock(model_id = "anthropic.claude-v2", region_name='us-east-1')

grounded = Groundedness(groundedness_provider=fclaude)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(fclaude.relevance_with_cot_reasons, name = "Answer Relevance").on_input_output()

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fclaude.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets.collect() .


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag, app_id='ChatGLM_500Chunk', feedbacks=[f_groundedness, f_qa_relevance, f_context_relevance])

In [14]:
tru.reset_database()

In [15]:
with tru_rag as recording:
    for question in questions:
        rag.query(question)

/home/ec2-user/miniconda3/envs/llm_notebook_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
App ChatGLM_500Chunk was not present in database. Adding it.


Groundedness per statement in source:   0%|          | 0/5 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

/home/ec2-user/miniconda3/envs/llm_notebook_env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ec2-user/miniconda3/envs/llm_notebook_env/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Multiple valid rating values found in the string: Statement Sentence: MoE模型的多任务指令微调性能优于单任务微调, Supporting Evidence: 当研究者们对 MoE 和对应性能相当的 T5 模型进行微调时,他们发现 T5 的对应模型表现更为出色。然而,当研究者们对 Flan T5 (一种 T5 的指令优化版本) 的 MoE 版本进行微调时,MoE 的性能显著提升。更值得注意的是,Flan-MoE 相比原始 MoE 的性能提升幅度超过了 Flan T5 相对于原始 T5 的提升,这意味着 MoE 模型可能从指令式微调中获益更多,甚至超过了稠密模型。此外,MoE 在多任务学习中表现更佳。与之前关闭辅助损失函数的做法相反,实际上这种损失函数可以帮助防止过拟合。, Score: 10


Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_id,,,,,
ChatGLM_500Chunk,0.8,0.981818,0.9025,7.5,0.0


In [ ]:
# with tru_recorder as recording:
#     for question in questions:
#         rag_chain.invoke(question)

In [17]:
tru.run_dashboard()
# tru.stop_dashboard() # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://172.31.2.24:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [10]:
from langchain.llms.bedrock import Bedrock
import sagemaker
import boto3

session = boto3.Session()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'
)

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.25,
    "top_k": 150,
    "top_p": 1,
}
modelId = 'anthropic.claude-v2'
claude_llm = Bedrock(
    model_id=modelId,
    client=bedrock_client,
    model_kwargs=inference_modifier,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [11]:
tt = 'ok'

In [12]:
claude_llm.invoke("GCP的Low C02 是什么意思？")

/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/mt_python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' GCP 的 Low C02 是指 Google Cloud Platform 提供的一种服务器配置,其特点是使用可再生能源,并且碳排放量较低。\n\n具体来说,Low C02 服务器使用的电力中至少包含了75%的可再生能源,如风能、太阳能等。相比普通服务器,这种服务器的碳排放量至少降低了30%。\n\n采用 Low C02 服务器可以帮助企业和组织降低其云计算相关的碳排放和环境影响。这对于一些注重可持续发展、环境保护的客户来说具有吸引力。\n\n使用 Low C02 服务器不会影响服务器的性能和功能,但会略微增加成本。总体来说,这是 GCP 提供的一个让云计算更环保的选择。企业可以根据自己的需求和可持续发展策略来决定是否采用这种服务器配置。'

In [13]:
from langchain.globals import set_verbose
set_verbose(True)

In [14]:
rag_chain.invoke("MOE有什么优势?")

'MOE（混合专家模型）相对于稠密模型的优势在于预训练速度更快和推理速度更快。同时，MOE需要的显存较少，且在微调方面存在诸多挑战。'

In [71]:
rag_chain.invoke("为什么MOE的训练和推理的速度更快?", )

<class 'langchain_core.documents.base.Document'>


'根据 Switch Transformers 论文中的描述，混合专家模型 (MoE) 通过将传统 Transformer 模型中的每个前馈网络 (FFN) 层替换为 MoE 层，其中 MoE 层由两个核心部分组成: 一个门控网络和若干数量的专家。这种替换可以提高训练和推理速度。具体来说，由于 MoE 模型只使用其中的一部分参数，所以在推理过程中只需要加载部分参数到内存中，从而降低了内存需求。此外，由于 MoE 模型中的专家数量相对较少，因此训练和推理过程中计算量更小，从而提高了速度。'

In [77]:
rag_chain.invoke("简单总结MoE模型的特点")

<class 'langchain_core.documents.base.Document'>


'MoE模型是一种混合专家模型的简写，它将传统Transformer模型中的每个前馈网络（FFN）层替换为MoE层，其中MoE层由两个核心部分组成：一个门控网络和若干数量的专家。MoE模型具有以下特点：1. 训练效率高，能够实现更高效的计算预训练；2. 与稠密模型相比，推理速度更快；3. 具有较好的泛化能力，但在微调阶段面临泛化能力不足的问题；4. 参数需求较大，推理过程中只使用其中一部分，需要将所有参数加载到内存中，因此对内存的需求较高。'

In [87]:
rag_chain.invoke("Shazeer将 MoE 应用在了哪个应用领域？")

<class 'langchain_core.documents.base.Document'>


'Shazeer 将 MoE 应用于自然语言处理（NLP）领域。'

In [91]:
rag_chain.invoke("MoE模型的训练过程中，如何保证令牌不会总是被发送给少数几个受欢迎的专家？")

'ST-MoE 论文中显示了哪些令牌组被发送给了哪个专家的表格。根据文中所述，混合专家模型 (MoE) 中的令牌会根据专家的权重比例被分配，以保证所有专家接收到大致相等数量的训练样本，从而平衡专家之间的选择。此外，辅助损失可以被用来鼓励给予所有专家相同的重要性，以避免令牌总是被发送给少数几个受欢迎的专家。'

In [92]:
rag_chain.invoke("MoE模型训练时，怎么保证token的负载均衡？")

'根据所提供的信息，MoE模型在训练时通过令牌路由和负载均衡机制来保证token的负载均衡。具体来说，该模型将令牌发送到拥有所需专家的节点，每个节点处理不同批次的训练样本。这种设计可以确保每个专家接收到不同数量的令牌，从而实现token的负载均衡。此外，该模型还采用了容量因子来提高模型性能，但这也意味着更高的通信成本和对保存激活值的显存的需求。在设备通信带宽有限的情况下，选择较小的容量因子可能是更佳的策略。'

In [93]:
rag_chain.invoke("GShard如何保证MoE模型的负载均衡？")

'GShard 保证 MoE 模型的负载均衡主要通过以下几个方面：\n1. 随机路由：在 Top-2 设置中，我们始终选择排名最高的专家，但第二个专家是根据其权重比例随机选择的。\n2. 专家容量：可以设定一个阈值，定义一个专家能处理多少令牌。如果两个专家的容量都达到上限，令牌就会溢出，并通过残差连接传递到下一层，或在某些情况下被完全丢弃。\n3. 辅助损失：为了鼓励给予所有专家相同的重要性，引入了一个辅助损失，确保所有专家接收到大致相等数量的训练样本，从而平衡了专家之间的选择。\n通过这些措施，GShard 能够实现 MoE 模型的负载均衡，从而提高训练效率。'

In [97]:
rag_chain.invoke("MoE的专家数量达到多少以后，效率就会显著降低？")

'根据所提供的信息，混合专家模型（MoE）的专家数量达到一定程度后，效率会显著降低。具体来说，当专家数量超过一定程度（约为1000-2000）时，训练和推理速度会明显变慢。因此，在选择专家数量时，需要权衡模型性能和计算资源的使用。'

In [99]:
chatglm_model("MoE的专家数量达到多少以后，效率就会显著降低？")

'这是一个比较复杂的问题，因为机器学习模型的效率受到很多因素的影响，包括数据集大小、模型结构、超参数选择等等。同时，专家数量也不是一个固定的值，而是需要根据具体问题进行调整。\n\n一般来说，当专家数量达到一定程度时，模型的效率可能会出现下降趋势。这是因为随着专家数量的增加，模型的复杂度也会增加，导致模型训练时间增加，同时过拟合的风险也会增加。但是，具体多少数量的专家会使得效率显著降低，需要根据具体问题和数据集情况进行实验和分析。\n\n另外，还有一些技巧可以帮助提高模型效率，例如正则化、早停、Dropout等。这些技巧可以在一定程度上减少过拟合现象，提高模型的泛化能力，同时也可以降低模型的复杂度，提高训练效率。'

In [100]:
rag_chain.invoke("在微调MoE这样的稀疏模型时，怎么防止出现过拟合？")

'根据ST-MoE论文，微调稀疏混合专家模型（MoE）时，可以通过以下方法防止过拟合：\n\n1. 降低学习率和调大批量可以提升稀疏模型微调质量。\n2. 尝试冻结所有非专家层的权重，实践中，这会导致性能大幅下降，但这符合预期，因为混合专家模型（MoE）层占据了网络的主要部分。可以尝试相反的方法：仅冻结MoE层的参数。实验结果显示，这种方法几乎与更新所有参数的效果相当。这种做法可以加速微调过程，并降低显存需求。\n3. 考虑特别的微调超参数设置，例如，稀疏模型往往更适合使用较小的批量大小和较高的学习率，这样可以获得更好的训练效果。'

In [102]:
rag_chain.invoke("怎么将MoE模型的推理阶段更加高效？")

'将MoE模型的推理阶段更加高效的方法包括：\n\n1. 优化门控网络和专家的权重训练过程，以提高模型在微调阶段的泛化能力。\n2. 采用更高效的计算方法，例如将稀疏混合专家模型 (SMoE) 蒸馏回具有更少实际参数但相似等价参数量的稠密模型。\n3. 探索合并专家模型的技术及其对推理时间的影响。\n4. 尝试对Mix'

In [103]:
rag_chain.invoke("模型的并行有几种形式？")

'问题: 模型的并行有几种形式？\n答案: 数据并行、模型并行、模型和数据并行、专家并行。'

In [104]:
rag_chain.invoke("部署 MoE 模型有哪些技术?")

'部署 MoE 模型有以下几种技术：\n\n1. 预先蒸馏 (Pre-distillation)：将 MoE 模型蒸馏回其对应的稠密模型，以保留稀疏性带来的性能提升，同时使得在推理中使用更小型的模型成为可能。\n\n2. 任务级别路由 (Task-level routing)：将整个句子或任务直接路由到一个专家，以提取出一个用于服务的子网络，有助于简化模型的结构。\n\n3. 专家网络聚合 (Expert network aggregation)：通过合并各个专家的权重，在推理时减少了所需的参数数量，在不显著牺牲性能的情况下降低模型的复杂度。'